# Bike Rentals Prediction - Regression Model

Build a model that predicts number of bike rentals on a particular day, given the parameters.

Table of contents:
1. Feature Engineering
2. Hyperparameters Optimization

In [ ]:
import os
os.chdir('..')

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import eli5
from eli5.xgboost import explain_weights_xgboost
import missingno as msno

import functions as f

np.random.seed(9)

In [2]:
def custom_rmse(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred[ y_pred < 0 ] = 0

    return np.sqrt(mean_squared_error(y_test, y_pred))

def custom_rmse_log(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred[ y_pred < 0 ] = 0

    return np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))

### Iterations summary

In order to keep track of progress, the result of each iteration is stored in the table below.


|iteration   |description   |result   | 
|---|---|---|
|0   |baseline model   |3259   |
|1   |all predictors, NA: -100   |2212   |
|2   |all predictors, NA: -100, start: 2016   |1951   |
|3   |all predictors, NA: -100, start: 2015   |1946   |
|4   |all predictors, NA: -100, start: 2015, season & holiday as dummy   |~~1911~~   |
|5   |all predictors, NA: -100, start: 2015, trends with log transform   |1945   |
|6   |all predictors, NA: -100, start: 2015, trends with log transform, month as dummy   |1922   |
|7   |all predictors, NA: -100, start: 2015, trends with log transform, month as dummy, if_fall   |~~1927~~   |
|8   |all predictors, NA: -100, start: 2015, trends with log transform, month as dummy, weather amplitude   |~~1937~~   |
|9   |all predictors, NA: -100, start: 2015, trends with log transform, month as dummy, month other   |1921   |
|10   |all predictors, NA: -100, start: 2015, trends with log transform, month as dummy, month other, limit predictors|1914   |
|11   |all predictors, NA: -100, start: 2015, trends with log transform, month as dummy, limit predictors   |1907
|12   |hyperparameters optimization   |1854

In [3]:
df = pd.read_parquet('data/data_predictors.parquet')
df['season'] = df['month'].apply(f.meteo_season)
#df.sample(3)

## Feature Engineering

### Fill NA and set minimum start date

- substitute all NaN values with -100 (number that doesn't occur in the dataset)
- set starting date for 2015-01-01 to exclude old, irrelevant records

In [4]:
df = df[df['start_date_'] > '2015-01-01']
df = df.fillna(-100)

### Prepare categorical variables

- Factorize: season and holiday
- Use one-hot encoding for month variable
- Tested variant: exclude *(Observed)* from holiday name - it didn't improve the result

In [5]:
df['season_f'] = pd.factorize(df['season'])[0]

#df['holiday'] = df['holiday'].str.replace(' (Observed)','', regex=False)
df['holiday_f'] = pd.factorize(df['holiday'])[0]
holidays_dict = pd.factorize(df['holiday'])[1]

In [6]:
df = pd.concat([df, pd.get_dummies(df['month'], prefix='month')], axis=1)

### Transform data

Use logaritmic transformation to re-calculate 'cb_trend' and 'bikerental_trend'

In [7]:
log_cols = ['cb_trend', 'bikerental_trend']

In [8]:
for i in range(len(log_cols)):
    col = log_cols[i]
    colname = col + '_log'
    df[colname] = np.log(df[col] + 0.00001)

C:\Users\Klaudia\Anaconda3\envs\bikerental\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


### Aplitude

Tested variant: Instead of using both min and max value, use aplitute (calculated and tested for all continuous weather-related features)

In [9]:
#ampl_features = ['wind', 'pressure']

#for feature in ampl_features:
#    df[feature + '_amp'] = df[feature + '_max'] - df[feature + '_min']

### Group less popular months

- select months with low feature importance and exclude them
- tested variant: replace months with low predictive value with 'other'

In [10]:
other_months = ['month_8', 'month_6', 'month_9', 'month_10', 'month_3', 'month_2', 'month_5', 'month_11', 'month_1', 'month_7', 'month12']
#df['month_other'] = df[other_months].sum(axis=1)

### Prepare X and y

- define target values (y)
- select columns containing numeric values
- exclude several columns from predictors' list (black list)

In [11]:
y = df['rentals_count_sum'].values

selected_features = list(df.select_dtypes(include=[np.number, np.bool]).columns)
black_list = ['rentals_count_sum', 'start_date', 'visits', 'wind_min', 'pressure_min'] + log_cols + other_months
selected_features = [feat for feat in selected_features if feat not in black_list]

X = df[selected_features].values

In [12]:
selected_features

['temperature_max',
 'temperature_min',
 'temperature_mean',
 'relative_temperature_max',
 'relative_temperature_min',
 'relative_temperature_mean',
 'wind_max',
 'wind_mean',
 'pressure_max',
 'pressure_mean',
 'rain_sum',
 'snow_sum',
 'thunder_sum',
 'clouds_cloudy_sum',
 'clouds_partly cloudy_sum',
 'clouds_clear_sum',
 'month',
 'weekday',
 'desktop_share',
 'visit_duration',
 'pages_visit',
 'bounce_rate',
 'pageviews_total',
 'season_f',
 'holiday_f',
 'month_4',
 'month_12',
 'cb_trend_log',
 'bikerental_trend_log']

### Train-Test split

- Divide data into train & test split.
- Use cross validation on training set to optimize model performance.
- Use test set to check final results of the model.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [14]:
model_xgboost = xgb.XGBRegressor(objective='reg:squarederror')
scores = cross_val_score(model_xgboost, X_train, y_train, cv=5, scoring=custom_rmse)
np.mean(scores), np.std(scores)

(1907.578565715249, 112.94135542061892)

The table below shows feature importances calculated based on average gain of the feature when it is used in trees. Temperature is very important predictor for the model. Other important features are: season, number of hours with thunder and trends from Google Trends for query: *Capital Bikeshare*.

In [15]:
model_xgboost.fit(X_train, y_train)
explain_weights_xgboost(model_xgboost, feature_names=selected_features, importance_type='gain', top=20)

Weight,Feature
0.3527,relative_temperature_mean
0.1688,temperature_mean
0.1057,temperature_max
0.0432,season_f
0.0349,thunder_sum
0.0274,cb_trend_log
0.0260,temperature_min
0.0231,clouds_cloudy_sum
0.0217,relative_temperature_min
0.0189,pageviews_total


## Hyperparameters tuning

There are two methods used:
- finding the best performing parameters with Python loop
- finding hyperparameters with hyperopt package

The first solution gave better results.

In [16]:
max_depth = np.arange(2,6)
min_child_weight = np.arange(1,11,2)
learning_rate = np.linspace(0.05,0.5,7)

max_score = 10000

for d in max_depth:
    for c in min_child_weight:
        for l in learning_rate:

            model_xgboost = xgb.XGBRegressor(max_depth=d, min_child_weight=c, learning_rate=l, objective='reg:squarederror')
            scores = cross_val_score(model_xgboost, X_train, y_train, cv=5, scoring=custom_rmse)
            
            new_score = np.mean(scores)
            if new_score < max_score:
                max_score = new_score
                print(d, c, l, new_score)

2 1 0.05 2065.0425611482656
2 1 0.125 1934.0995256538718
2 1 0.2 1890.9982330776704
2 1 0.35 1883.0311014833096
2 3 0.27499999999999997 1854.4580181244448


In [17]:
from functools import partial
from hyperopt import hp
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [18]:
def objective(space):
    
    xgb_params = {
        'max_depth': int(space['max_depth']),
        'colsample_bytree': space['colsample_bytree'],
        'learning_rate': space['learning_rate'],
        'subsample': space['subsample'],
        #'seed': int(space['seed']),
        'min_child_weight': int(space['min_child_weight']),
        'reg_alpha': space['reg_alpha'],
        'reg_lambda': space['reg_lambda'],
        'n_estimators': 100
    }

    scores = cross_val_score(xgb.XGBRegressor(objective='reg:squarederror', **xgb_params), X, y, cv=3, scoring=custom_rmse)
    mean_score = np.mean(scores)
    
    print("mean score: {0}, std score: {1}".format(mean_score, np.std(scores)))
    
    return{'loss':mean_score, 'status': STATUS_OK }
    
space ={
    'max_depth': hp.quniform ('x_max_depth', 1, 10, 1),
    'colsample_bytree': hp.uniform ('x_colsample_bytree', 0.8, 1.),
    'learning_rate': hp.uniform ('x_learning_rate', 0.05, 0.3),
    'subsample': hp.uniform ('x_subsample', 0.7, 1.),
    #'seed': hp.quniform ('x_seed', 0, 10000, 50),
    'min_child_weight': hp.quniform ('x_min_child_weight', 1, 10, 1),
    'reg_alpha': hp.loguniform ('x_reg_alpha', 0., 1.),
    'reg_lambda': hp.uniform ('x_reg_lambda', 0.7, 1.),
}


trials = Trials()
best_params = fmin(fn=objective,
            space=space,
            algo=partial(tpe.suggest, n_startup_jobs=5),
            max_evals=15,
            trials=trials)

print("The best params: ", best_params)

mean score: 2043.9683628434734, std score: 142.6087728849027                                                           
mean score: 1995.584938339801, std score: 187.4112813357665                                                            
mean score: 1995.6114160671816, std score: 165.0929515967763                                                           
mean score: 2076.1814189719594, std score: 174.90806605172241                                                          
mean score: 2187.4810815078695, std score: 153.48757492645825                                                          
mean score: 2207.8372665853863, std score: 284.2234863636246                                                           
mean score: 2064.2773592603116, std score: 198.13255109736392                                                          
mean score: 2118.707334864782, std score: 150.39107100401696                                                           
mean score: 2116.5271058632, std score: 

## Train & Evaluate Final Model

Train final model on train set and check prediction on test set.

In [19]:
model_xgboost = xgb.XGBRegressor(max_depth=2, min_child_weight=3, learning_rate=0.27499999999999997, objective='reg:squarederror')
model_xgboost.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.27499999999999997,
             max_delta_step=0, max_depth=2, min_child_weight=3, missing=None,
             n_estimators=100, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [20]:
custom_rmse(model_xgboost, X_test, y_test)

1516.2612761731884

In [63]:
random_choice = np.random.choice(X_test.shape[0],1) #np.array([238])

In [64]:
df_random_choice = pd.DataFrame(X_test[random_choice], columns=selected_features)

Information about input and predicted bike rentals number:

In [65]:
print('Month: %d' % df_random_choice['month'][0])
print('Weekday (0-Mon, 6-Sun): %d' % df_random_choice['weekday'][0])
print('Min temperature: %d' % df_random_choice['temperature_min'][0])
print('Max temperature: %d' % df_random_choice['temperature_max'][0])
print('Hours of rain: %d' % df_random_choice['rain_sum'][0])
print('Holiday: %d' % holidays_dict[df_random_choice['holiday_f'][0]])

Month: 10
Weekday (0-Mon, 6-Sun): 0
Min temperature: 13
Max temperature: 23
Hours of rain: 5
Holiday: -100


In [66]:
print('number of bike rentals ')
print('predicted: %d' % model_xgboost.predict(X_test[random_choice])[0])
print('actual: ', y_test[random_choice][0])

number of bike rentals 
predicted: 10348
actual:  10208
